In [10]:
# import packages
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras import backend as K
import keras
import keras_metrics
import tensorflow as tf    
from tensorflow import keras 
import keras_metrics as km

In [11]:
# load the package
final_df = pd.read_csv('/project/train_data.csv')

In [12]:
final_df.head()

,Unnamed: 0,abstract.text_sw,biblio.invention_title.text_sw,claims.claims_sw,yo2
0,98980,disclosure relates hand control input device c...,unknown,a hand control input device comprising a ...,0
1,69824,one aspect present invention directed plant pl...,bacterial spore compositions for industrial uses,a plant or plant part coated with a composi...,1
2,9928,demonstrative embodiments include apparatuses ...,apparatus system and method of setting up an ...,a product comprising one or more tangible c...,0
3,75599,wavelength multiplexing apparatus includes div...,wavelength multiplexing apparatus and method,a wavelength multiplexing apparatus compris...,0
4,95621,first second inputs received first input indic...,method and apparatus for transmitter optimizat...,canceled a system comprising one ...,0


In [13]:
final_df.shape

(75000, 5)

In [14]:
# split X and y
y = final_df['yo2']
X = final_df.drop(columns = 'yo2')
X['tac'] = X['biblio.invention_title.text_sw'] + ' ' + X['abstract.text_sw'] + ' ' + X['claims.claims_sw']
X['title_abs'] = X['biblio.invention_title.text_sw'] + ' ' + X['abstract.text_sw'] 
X['title_claims'] = X['biblio.invention_title.text_sw'] + ' ' + X['claims.claims_sw']
X['abs_claims'] = X['abstract.text_sw'] + ' ' + X['claims.claims_sw']
X['abstract'] = X['abstract.text_sw']
X['title'] = X['biblio.invention_title.text_sw']
X['claims'] = X['claims.claims_sw']

In [15]:
# split train and test
trainX, testX, trainy, testy = train_test_split(X['claims'], y, test_size=0.25, random_state = 42)

In [16]:
trainX.shape,trainy.shape,testX.shape,testy.shape

((56250,), (56250,), (18750,), (18750,))

In [17]:
from pickle import dump

def save_dataset(dataset, filename):
    dump(dataset, open(filename, 'wb'))
    print('Saved: %s' % filename)
    
save_dataset([trainX,trainy], 'train_claims.pkl') 

Saved: train_claims.pkl


In [18]:
import csv
import pydot
filename = 'train_claims.pkl'

In [19]:
# load a clean dataset
def load_dataset(filename):
    return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])

# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=3, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(500, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision(),
                                                                         tf.keras.metrics.Recall(),
                                                                        tf.keras.metrics.AUC(curve='ROC')])
    # summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel_claims.png')
    return model

In [ ]:
# load training dataset
trainLines, trainLabels = load_dataset(filename)

# create tokenizer
tokenizer = create_tokenizer(trainLines)

# calculate max document length
length = max_length(trainLines)

# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# encode data
trains_X = encode_text(tokenizer, trainLines, length)
print(trains_X.shape)

# define model
model = define_model(length, vocab_size)

class_weights = 1/(trainLabels.value_counts()/len(trainLabels))
class_weights = {0: class_weights[0], 1: class_weights[1]}

# fit model
history = model.fit([trains_X,trains_X,trains_X], array(trainLabels), epochs=10, batch_size=16, class_weight=class_weights)

# save the model
model.save('model_claims.h5')

Max document length: 20826
Vocabulary size: 94492
(56250, 20826)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20826)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 20826)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 20826)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20826, 100)   9449200     input_1[0][0]                    
_____________________________

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



3516/3516 [==============================] - 9453s 3s/step - loss: 0.2244 - precision: 0.7410 - recall: 0.9611 - auc: 0.9912
Epoch 6/10
3516/3516 [==============================] - 12288s 3s/step - loss: 0.2494 - precision: 0.7342 - recall: 0.9597 - auc: 0.9889
Epoch 7/10
3516/3516 [==============================] - 17173s 5s/step - loss: 0.2347 - precision: 0.7390 - recall: 0.9597 - auc: 0.9898
Epoch 8/10
3516/3516 [==============================] - 19290s 5s/step - loss: 0.3186 - precision: 0.7057 - recall: 0.9464 - auc: 0.9840
Epoch 9/10
1738/3516 [=============>................] - ETA: 1:13:52 - loss: 0.1863 - precision: 0.7848 - recall: 0.9723 - auc: 0.9936

In [ ]:
#list all data in history
print(history.history.keys())

# summarize history for loss
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()